In [1]:
import os
import pickle
import csv
import numpy as np

from sklearn.linear_model import OrthogonalMatchingPursuit, OrthogonalMatchingPursuitCV
from sklearn.pipeline import make_pipeline

#### Функция чтения набора данных

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='H_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for pr in pr_list:
                pr_arr_row.append(row[pr])
            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name] if row[y_name] else 0)
    X = np.asarray(pr_arr, dtype=np.float64)
    y = np.asarray(y_arr, dtype=np.float64)
    return X, y

#### Функция формирования тестового набора данных с подстановкой нормированных значений

In [3]:
def test_norm(x, pr_list, norms):
    x_norm = np.copy(x)
    for col, pr in enumerate(pr_list):
        if pr in norms:
            x_norm[:, col:col+1] = norms[pr]
    return x_norm

In [4]:
def write_dataset_csv(year, dataset, dataset_name, fieldnames, pr_group, mode='training'):
    if mode == 'estimation':
        dir_path = f'results/Estimation/{year}/{dataset_name}/group-{pr_group}/'
        file_name = f'{dataset_name}-гр{pr_group}-Оценка.csv'
    elif mode == 'training':
        dir_path = f'results/Models/{year}/'
        file_name = f'{dataset_name}-гр{pr_group}-Обучение.csv'
    elif mode == 'forecast':
        dir_path = f'results/Forecast/{year}/'
        file_name = f'{dataset_name}-гр{pr_group}-Прогноз.csv'
    else:
        ...
    
    with open(
        f'{dir_path}'
        f'{file_name}', 
        'w', newline='', encoding='utf-8'
    ) as csvfile:
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, 
                                delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


### Функция прогнозирования

In [5]:
def forecast(year, norms=False):

    fieldnames = [
        'Predictors', 
        'Equations', 
        'Method', 
        'Criterion', 
        'Correlation', 
        'Pm',
        'R2',        
        'R2_t',
        'Prediction',
    ]
    
    predict_data_dir = f'data/{year}/Predict'
    
    models_dir = f'results/Models/{year}'

    # Получить список файлов .ipnb из results\Models\<year>
    file_list = tuple(filter(lambda fn: '.pickle' in fn, os.listdir(models_dir)))
    # print(file_list)

    # Сделать множество из кортежей (Название-датасета, группа), преобразовать в список, отсортировать
    dataset_group = set()
    for fn in file_list:
        dataset, year, group, model = fn.split('_')
        dataset_group |= {(dataset, group.split('гр')[1])}
    ds_list = sorted(dataset_group)
    # print(ds_list)

    # Для каждого элемента списка создать result_list[one_model_raw: dict]
    
    for ds, group in ds_list:
        result_list = []
        ds_models = filter(lambda fn: ds in fn and group in fn, file_list)
        for file_name in ds_models:
            with open(f'results/Models/{year}/{file_name}', 'rb') as f:
                model_info = pickle.load(f)
                model = model_info['Model_full']
                #model = model_info['Model_train']
                
                # Прочитать новые признаки (предикторы) для прогноза
                X_new, y = get_river_dataset(f'{predict_data_dir}/{ds}.csv', pr_list=model_info['Predictors_list'])

                # Подстановка нормированных значений для целей тестирования
                if norms:
                    # Подстановка норм в исходный набор признаков
                    X_new = test_norm(X_new, model_info['Predictors_list'], model_info['Norms_data'])

                print(model_info['Dataset_name'])
                print(model_info['Predictors_list'])
                print(model_info['Norms_data'])
                print(model_info['Method'])
                print("y")
                print(y)
                print(f'R2={model_info["R2"]}, R2_t={model_info["R2_t"]}')
                
                y_predicted = np.ravel(model.predict(X_new))
                print("y_predicted")
                print(y_predicted)
                print('X_new')
                print(X_new)
                print(model)
                model_info['Prediction'] = y_predicted[-1]
                result_list.append(model_info)
                # Сортировка результатов по каждому датасету
                result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))
                
                # Запись в .csv файл
                write_dataset_csv(year, result_list, ds, fieldnames, pr_group=group, mode='forecast')
                print('------------------------------------------------------------------------------')
    
    

#### Запуск процесса прогнозирования

In [6]:
forecast(2023, norms=True)

Вилия-Михалишки
['S_2802', 'Smax', 'H_2802', 'X', 'X1', 'X2', 'X3', 'Xs', 'L_max', 'L_2802', 'Q12', 'Q01', 'Q02', 'Y_sum']
{'S_max': 60.0, 'X': 116.0, 'X1': 46.0, 'X2': 37.0, 'L_max': 57.0}
KNeighborsRegressor
y
[260. 284. 369. 371. 133. 189. 136. 130. 416. 270. 189. 328. 149. 264.
 302. 185. 248. 192. 164. 134. 439. 172. 207. 320. 123. 238. 202. 133.
 172. 237.   0.]
R2=0.6943499370052297, R2_t=0.7597197193585816
y_predicted
[316.29214272 303.52217425 344.37727224 299.60333461 172.91114482
 188.56471713 109.00969356 128.63002186 242.68018182 243.84503201
 139.77548248 321.52849442 150.24018394 289.77065017 228.32123136
 144.5460587  247.1690795  181.78430894 108.00250926 131.38910009
 424.92678758 173.44273002 181.58509424 239.35106671 135.99016136
 242.85711936 214.35358776 134.38921098 178.98071121 281.16797026
 230.18783579]
X_new
[[ 82.    90.   123.   116.    46.    37.   143.   112.    57.    71.
   38.5   27.9   30.8   24.5 ]
 [ 88.    92.   156.   116.    46.    37.   192.   1